In [1]:
import random
inW	= ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]
wheel = [["E","K","M","F","L","G","D","Q","V","Z","N","T","O","W","Y","H","X","U","S","P","A","I","B","R","C","J"],["A","J","D","K","S","I","R","U","X","B","L","H","W","T","M","C","Q","G","Z","N","P","Y","F","V","O","E"],["B","D","F","H","J","L","C","P","R","T","X","V","Z","N","Y","E","I","W","G","A","K","M","U","S","Q","O"],["E","S","O","V","P","Z","J","A","Y","Q","U","I","R","H","X","L","N","F","T","G","K","D","C","M","W","B"],["V","Z","B","R","G","I","T","Y","U","P","S","D","N","H","L","X","A","W","M","J","Q","O","F","E","C","K"]]
reflec = ["Y","R","U","H","Q","S","L","D","P","X","N","G","O","K","M","I","E","B","F","Z","C","W","V","J","A","T"]
change = [17,5,22,10,0]
inC = {}
for i in range(len(inW)):
	inC[inW[i]] = i

class enigma:
	def __init__(self,wheels,offset,swap=None):
		self.wh = []
		self.revert = [{},{},{}]
		self.flip = []
		self.off = []
		for i in wheels:
			self.flip.append(change[i])
			tmp = []
			for j in wheel[i]:
				tmp.append(j)
			self.wh.append(tmp)
		for i in range(26):
			for j in range(3):
				self.revert[j][self.wh[j][i]] = i
		if offset is not None:
			for k in offset:
				self.off.append(0+k)
		else:
			self.off = [0,0,0]
# 		print(self.off)
		self.swap = {}
		if swap != None:
			for i in swap:
				self.swap[i] = swap[i]
				self.swap[swap[i]] = i
	def getChar(self,x):
		if x in self.swap:
			x = self.swap[x]
		self.off[0] = (self.off[0] + 1)%26
# 		print("SpinWheel A")
		if self.off[0] == self.flip[0]:
# 			print("SpinWheel B")
			self.off[1] = (self.off[1] + 1)%26
			if self.off[1] == self.flip[1]:
# 				print("SpinWheel C")
				self.off[2] = (self.off[2] + 1)%26
		elif (self.off[1]+1)%26 == self.flip[1]:
			self.off[1] = (self.off[1]+1)%26
			self.off[2] = (self.off[2]+1)%26
		for i in range(3):
			x = inW[(inC[self.wh[i][(inC[x]+self.off[i])%26]]-self.off[i])%26]
			# print(x)
		x = reflec[inC[x]]
		# print(x)
		for i in [2,1,0]:
			x = inW[(self.revert[i][inW[(inC[x]+self.off[i])%26]]-self.off[i])%26]
		if x in self.swap:
			x = self.swap[x]
		return x
	def getStr(self, String):
		re = ""
		String = String.upper()
		for i in String:
			re = re+(self.getChar(i))
		return re

In [15]:
# Simulate encription
SecureWheel = [0,1,2,3,4]
SecureOffset = []
SecureSwap = {}
random.shuffle(SecureWheel)
SecureWheel.pop(0)
SecureWheel.pop(0)
for i in range(3):
	SecureOffset.append(random.randint(0,25))
SecureSwaped = []
for i in range(10):
	a = inW[random.randint(0,25)]
	while a in SecureSwaped:
		a = inW[random.randint(0,25)]
	SecureSwaped.append(a)
	b = inW[random.randint(0,25)]
	while b in SecureSwaped:
		b = inW[random.randint(0,25)]
	SecureSwaped.append(b)
	SecureSwap[a] = b
print("before Encripte Wheel: %s\tOffset: %s"%(SecureWheel,SecureOffset))
print("swap set:")
for i in SecureSwap:
    print("%s<->%s"%(i,SecureSwap[i]))

#Encripte Key

passW = "WEATHERREPORTOFTHEDAY"
# for i in range(10):
#     tmpStr = inW[random.randint(0,25)] + inW[random.randint(0,25)] + inW[random.randint(0,25)]
#     tmpStr = tmpStr+tmpStr
#     passW.append(tmpStr)
print("Keys:          %s"%(passW))

tmpEn = enigma(wheels = SecureWheel, offset = SecureOffset,swap=SecureSwap)
encriptPassW = tmpEn.getStr(passW)
print("Encripted Key: %s"%(encriptPassW))

before Encripte Wheel: [2, 4, 3]	Offset: [6, 2, 3]
swap set:
O<->G
L<->S
N<->V
Q<->E
X<->D
P<->U
K<->C
T<->A
M<->W
J<->I
Keys:          WEATHERREPORTOFTHEDAY
Encripted Key: QVIOOHHOOAWYYEYVOTPDI


In [16]:
# Calc Loop
assert(len(encriptPassW)==len(passW))
tmpDict = {}
for i in range(len(passW)):
    if passW[i] not in tmpDict:
        tmpDict[passW[i]] = [(encriptPassW[i],i)]
    else:
        tmpDict[passW[i]].append((encriptPassW[i],i))
for i in range(len(encriptPassW)):
    if encriptPassW[i] not in tmpDict:
        tmpDict[encriptPassW[i]] = [(passW[i],i)]
    else:
        tmpDict[encriptPassW[i]].append((passW[i],i))
# print(tmpDict)
dicts = [{}]
for i in tmpDict:
    tdict = []
    for j in tmpDict[i]:
        tmdict = []
        for k in dicts:
            l = {}
            for o in k:
                l[o] = k[o]
            tmdict.append(l)
        for k in tmdict:
            k[i] = j
        tdict.extend(tmdict)
#     print("after %s: %s"%(i,tdict))
    dicts = tdict
loops = []
found = []
for d in dicts:
    done = []
    for i in d:
        recur = []
        j = i
        isConti = True
        while i in d and isConti:
            recur.append(d[i][1])
            if i in done:
                isConti = False
                continue
            done.append(i)
            i = d[i][0]
            if d[i][1] == recur[len(recur)-1]:
                break
            if i == j:
                isConti = False
                if str(sorted(list(recur))) not in found:
                    loops.append(recur)
                    found.append(str(sorted(list(recur))))
print("Loops:\n%s"%(loops))

Loops:
[[19, 18, 9], [17, 15, 1], [5, 4, 13], [8, 13], [17, 3, 13], [5, 16, 13], [5, 6, 7, 13], [5, 4, 3, 15, 1], [8, 3, 15, 1], [13, 3, 15, 1], [5, 4, 3, 17], [8, 3, 17], [5, 16, 3, 15, 1], [5, 16, 3, 17], [5, 6, 7, 3, 15, 1], [5, 6, 7, 3, 17], [16, 4], [8, 4, 5], [6, 7, 4], [16, 7, 6], [8, 7, 6, 5], [5, 16, 8], [17, 12, 11, 6, 5], [17, 12, 11, 6, 4, 13], [17, 12, 11, 6, 16, 13], [17, 12, 11, 7, 13], [12, 11, 6, 4, 3], [12, 11, 6, 16, 3], [8, 3, 12, 11, 6, 5], [13, 3, 12, 11, 6, 5], [12, 11, 7, 3], [17, 12, 11, 7, 4, 5], [17, 12, 11, 6, 4, 8], [17, 12, 11, 6, 16, 8], [17, 12, 11, 7, 8], [17, 12, 11, 7, 16, 5], [5, 6, 11, 12, 15, 1], [8, 4, 6, 11, 12, 15, 1], [13, 4, 6, 11, 12, 15, 1], [5, 4, 7, 11, 12, 15, 1], [8, 7, 11, 12, 15, 1], [13, 7, 11, 12, 15, 1], [5, 16, 7, 11, 12, 15, 1], [8, 16, 6, 11, 12, 15, 1], [13, 16, 6, 11, 12, 15, 1]]


In [17]:
# Crack the PassWord With Turing's Method
possW = []
possO = []
for a in range(5):
    for b in range(5):
        if a==b:
            continue
        for c in range(5):
            if c==a or c==b:
                continue
            tmpW = [a,b,c]
            for d in range(26):
                for e in range(26):
                    for z in range(26):
                        tmpO = [d,e,z]
                        eni = enigma(wheels = tmpW,offset=tmpO)
                        eniDict = {}
                        for l in range(len(encriptPassW)):
                            eniDict[l] = list(eni.off)
                            eni.getChar('A')
                        wrong = False
                        for loop in loops:
                            if wrong:
                                break
                            hasMatch = False
                            for f in range(26):
                                tmpLetter = inW[f]
                                for i in loop:
                                    eni.off = list(eniDict[i])
                                    tmpLetter = eni.getChar(tmpLetter)
                                if tmpLetter == inW[f]:
                                    hasMatch = True
                                    break
                            if not hasMatch:
                                wrong = True
                        if not wrong:
                            possW.append(tmpW)
                            possO.append(tmpO)
# print(possW)
print("possible results:")
#''',sorted(guessSwapped[i].items(), key=lambda d: d[1],reverse=True)'''
for i in range(len(possW)):
	print("wheels: %s\toffsets: %s\n"%(possW[i],possO[i]))

possible results:
wheels: [2, 4, 3]	offsets: [6, 2, 3]



In [12]:
print(len(possW))

47776


In [58]:
import copy
possW = [[2, 4, 3]]
possO = [[6, 2, 3]]
passW = "WEATHERREPORTOFTHEDAY"
encriptPassW = "QVIOOHHOOAWYYEYVOTPDI"
# Calc Loop
assert(len(encriptPassW)==len(passW))
tmpDict = {}
for i in range(len(passW)):
    if passW[i] not in tmpDict:
        tmpDict[passW[i]] = [(encriptPassW[i],i)]
    else:
        tmpDict[passW[i]].append((encriptPassW[i],i))
for i in range(len(encriptPassW)):
    if encriptPassW[i] not in tmpDict:
        tmpDict[encriptPassW[i]] = [(passW[i],i)]
    else:
        tmpDict[encriptPassW[i]].append((passW[i],i))
# print(tmpDict)
dicts = [{}]
for i in tmpDict:
    tdict = []
    for j in tmpDict[i]:
        tmdict = []
        for k in dicts:
            l = {}
            for o in k:
                l[o] = k[o]
            tmdict.append(l)
        for k in tmdict:
            k[i] = j
        tdict.extend(tmdict)
#     print("after %s: %s"%(i,tdict))
    dicts = tdict
loops = []
found = []
loopInChar = {}
for d in dicts:
    done = []
    for i in d:
        recur = []
        j = i
        loopString = ""
        isConti = True
        while i in d and isConti:
            recur.append(d[i][1])
            if i in done:
                isConti = False
                continue
            done.append(i)
            loopString = loopString + i
            i = d[i][0]
            if d[i][1] == recur[len(recur)-1]:
                break
            if i == j:
                isConti = False
                if str(recur) not in loopInChar:
                    loopInChar[str(recur)] = []
                if loopString not in loopInChar[str(recur)]:
                    loopInChar[str(recur)].append(loopString)
                if str(sorted(list(recur))) not in found:
                    loops.append(recur)
                    found.append(str(sorted(list(recur))))
print(loopInChar)
saveLoopChar = copy.deepcopy(loopInChar)

{'[19, 18, 9]': ['ADP'], '[17, 15, 1]': ['ETV'], '[5, 4, 13]': ['EHO'], '[8, 13]': ['EO'], '[17, 3, 13]': ['ETO'], '[5, 16, 13]': ['EHO'], '[5, 6, 7, 13]': ['EHRO'], '[5, 4, 3, 15, 1]': ['EHOTV'], '[8, 3, 15, 1]': ['EOTV'], '[13, 3, 15, 1]': ['EOTV'], '[5, 4, 3, 17]': ['EHOT'], '[8, 3, 17]': ['EOT'], '[13, 3, 17]': ['EOT'], '[5, 16, 3, 15, 1]': ['EHOTV'], '[5, 16, 3, 17]': ['EHOT'], '[5, 6, 7, 3, 15, 1]': ['EHROTV'], '[5, 6, 7, 3, 17]': ['EHROT'], '[16, 4]': ['HO'], '[8, 4, 5]': ['EOH'], '[17, 3, 4, 5]': ['ETOH'], '[13, 4, 5]': ['EOH'], '[6, 7, 4]': ['HRO'], '[4, 7, 6]': ['HOR'], '[16, 7, 6]': ['HOR'], '[8, 7, 6, 5]': ['EORH'], '[17, 3, 7, 6, 5]': ['ETORH'], '[13, 7, 6, 5]': ['EORH'], '[5, 4, 8]': ['EHO'], '[17, 3, 8]': ['ETO'], '[13, 8]': ['EO'], '[5, 16, 8]': ['EHO'], '[5, 6, 7, 8]': ['EHRO'], '[4, 16]': ['HO'], '[8, 16, 5]': ['EOH'], '[17, 3, 16, 5]': ['ETOH'], '[13, 16, 5]': ['EOH'], '[6, 7, 16]': ['HRO'], '[9, 18, 19]': ['APD'], '[17, 12, 11, 6, 5]': ['ETYRH'], '[17, 12, 11, 6, 4,

In [90]:
loopInChar = copy.deepcopy(saveLoopChar)
for w in possW:
    for o in possO:
        possLoop = {}
        tmpW = w
        tmpO = o
        eni = enigma(wheels = tmpW,offset=tmpO)
        eniDict = {}
        for l in range(len(encriptPassW)):
            eniDict[l] = list(eni.off)
            eni.getChar('A')
        wrong = False
        for loop in loops:
            hasMatch = False
            for f in range(26):
                tmpLetter = inW[f]
                loopPhrase = ""
                for i in loop:
                    eni.off = list(eniDict[i])
                    loopPhrase = loopPhrase + tmpLetter
                    tmpLetter = eni.getChar(tmpLetter)
                if tmpLetter == inW[f]:
                    hasMatch = True
                    if str(loop) not in possLoop:
                        possLoop[str(loop)] = []
                    if loopPhrase not in possLoop:
                        possLoop[str(loop)].append(loopPhrase)
            if not hasMatch:
                print("Error")
        notSwap = []
        swap = {}
        print(possLoop)
        for q in possLoop:
#             print(q)
            if len(possLoop[q]) == 1:
                for i in range(len(possLoop[q][0])):
                    if possLoop[q][0][i] == loopInChar[q][0][i]:
                        if possLoop[q][0][i] not in notSwap:
                            notSwap.append(possLoop[q][0][i])
                        if possLoop[q][0][i] in swap:
#                             print(possLoop[q][0][i])
                            print("notMatch")
                    elif possLoop[q][0][i] not in notSwap and loopInChar[q][0][i] not in notSwap:
                        swap[possLoop[q][0][i]] = loopInChar[q][0][i]
                        swap[loopInChar[q][0][i]] = possLoop[q][0][i]
        for place in range(100):
            finished = []
            for q in possLoop:
                if q in finished:
                    continue
                if len(loopInChar[q]) < 1:
                    continue
                if len(loopInChar[q]) > 1:
                    continue
                if len(possLoop[q]) == 1:
                    for i in range(len(possLoop[q][0])):
                        if possLoop[q][0][i] == loopInChar[q][0][i]:
                            if possLoop[q][0][i] not in notSwap:
                                notSwap.append(possLoop[q][0][i])
                            if possLoop[q][0][i] in swap:
    #                             print(possLoop[q][0][i])
                                print("notMatch")
                        elif possLoop[q][0][i] not in notSwap and loopInChar[q][0][i] not in notSwap:
                            swap[possLoop[q][0][i]] = loopInChar[q][0][i]
                            swap[loopInChar[q][0][i]] = possLoop[q][0][i]
                    finished.append(q)
                    loopInChar[q].pop(0)
                    possLoop[q].pop(0)
                    continue
                if len(possLoop[q]) > 1:
                    delete = []
                    for jloop in possLoop[q]:
                        for i in range(len(jloop)):
                            if jloop[i] in notSwap:
                                if jloop[i]!=loopInChar[q][0][i]:
                                    delete.append(jloop)
                                    break
                            elif jloop[i] in swap:
                                if swap[jloop[i]]!=loopInChar[q][0][i]:
                                    delete.append(jloop)
                                    break
                    for k in delete:
                        possLoop[q].remove(k)
                    if len(possLoop[q]) == 0:
                        finished.append(q)
        foundAll = False
        for l in range(10):
            if foundAll:
                break
            foundAll = True
            completeSet = []
            for i in swap:
                completeSet.append(i)
            for i in notSwap:
                completeSet.append(i)
            eni = enigma(tmpW,tmpO,swap)
            Strs = [eni.getStr(encriptPassW),passW]
            print("%s\n%s"%(Strs[0],Strs[1]))
            eni = enigma(tmpW,tmpO,swap)
            Strs.extend([eni.getStr(passW),encriptPassW])
            print("%s\n%s"%(Strs[2],Strs[3]))
            print("%s Sets Left"%((26-(len(completeSet) - len(notSwap)+6))/2))
            print()
            for i in range(len(Strs[0])):
                if Strs[0][i]!=Strs[1][i] or Strs[2][i]!=Strs[3][i]:
                    foundAll = False
                    if Strs[0][i] in completeSet and Strs[1][i] in completeSet:
                        swap[Strs[2][i]] = Strs[3][i]
                        swap[Strs[3][i]] = Strs[2][i]
                        break
                    if Strs[2][i] in completeSet and Strs[3][i] in completeSet:
                        swap[Strs[0][i]] = Strs[1][i]
                        swap[Strs[1][i]] = Strs[0][i]
                        break
        print("Doing last trys to fix more replacements")
        print(swap)
        print(notSwap)

{'[19, 18, 9]': ['HWM', 'TXU', 'WHQ'], '[17, 15, 1]': ['GUW', 'IXY', 'KVD', 'QAN'], '[5, 4, 13]': ['QHG'], '[8, 13]': ['GQ', 'QG'], '[17, 3, 13]': ['BJV', 'GUQ', 'NTI', 'QAG'], '[5, 16, 13]': ['KMX', 'QHG'], '[5, 6, 7, 13]': ['QHRG'], '[5, 4, 3, 15, 1]': ['QHGAN'], '[8, 3, 15, 1]': ['GQUW', 'QGAN', 'VUQE'], '[13, 3, 15, 1]': ['GQUW', 'QGAN', 'RAGF'], '[5, 4, 3, 17]': ['QHGA', 'RYWM'], '[8, 3, 17]': ['GQU', 'QGA'], '[5, 16, 3, 15, 1]': ['QHGAN'], '[5, 16, 3, 17]': ['QHGA'], '[5, 6, 7, 3, 15, 1]': ['QHRGAN'], '[5, 6, 7, 3, 17]': ['QHRGA'], '[16, 4]': ['GH', 'HG'], '[8, 4, 5]': ['EFS', 'OXN', 'QGH'], '[6, 7, 4]': ['DBT', 'HRG'], '[16, 7, 6]': ['HGR'], '[8, 7, 6, 5]': ['DKNF', 'QGRH'], '[5, 16, 8]': ['QHG'], '[17, 12, 11, 6, 5]': ['CDLEZ', 'OLDFN', 'QAYRH'], '[17, 12, 11, 6, 4, 13]': ['AQUTUR', 'QAYRHG', 'TNJVYW'], '[17, 12, 11, 6, 16, 13]': ['IXSWKN', 'LOFDBP', 'QAYRHG'], '[17, 12, 11, 7, 13]': ['KVTUX', 'QAYRG'], '[12, 11, 6, 4, 3]': ['AYRHG'], '[12, 11, 6, 16, 3]': ['AYRHG'], '[8, 3, 12

In [22]:
print(possW)

[[2, 4, 3]]
